In [1]:
import numpy as np 
import scipy.stats as ss
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
def YTM (pv: float, maturity: int, cashflow: float = 100):
    """
    This function calculates the yield-to-maturity (ytm) of the promised cash flow
    :param pv: the promised cash flow of the annuity where the calculated value of this promised cash flow
    :param cashflow: the provided cashflow per each coupon data
    :param maturity: the maturity of the provided fixed income asset in months (e.g. 3yr = 36 months)
    return: The ytm is calculated as a continuously compounding annual rate
    """
    
    ytm = 0.0
    step = 0.01

    # Calculate the ytm to within 0.000001, i.e., 0.0001% or 0.01 bps
    while (step > 0.000001):

        # Keep increasing the trial ytm until the resulting present value (pv0) is too small (less than pv).
        while (True):

            # Calculate the present value with a discount rate of "ytm".
            pv0 = 0
            for m in range(1, maturity+1):
                pv0 += cashflow * np.math.exp(-ytm * m / 12.0)

            # Is ytm now too big? If not, increase it further by the amount "step".
            if (pv0 > pv):
                ytm += step

            # If so, reduce it by "step" and break out of the "while (1)" loop. Reduce the step size for the next iteration.
            else:
                ytm -= step
                step /= 10.0
                break

    #return the yield-to-maturity as a percent.
    return 100.0 * ytm              

In [7]:
credit_rating = {"AAA":0.035, "AA":0.04, "A":0.045, "BBB":0.05, "Speculative":0.06}

In [195]:
def cash_waterfall(cashflow:int, credit_ratings:dict, index=0):
    # promised cash flow per CDO
    ideal_cash = (100 * 20) / 5 
    
    ratings = ["AAA", "AA", "A", "BBB", "Speculative"]
    cash = 0 
    defaults = 0
    
    for i in ratings:
        # if the cashflow is greate than the ideal cash flow we allocate the cash and reduce the cash pool
        if cashflow > ideal_cash:
            cash = ideal_cash
            cashflow -= ideal_cash
        else:
            # if you lack sufficient cash, then remaining tranches recieve no money
            if defaults == 0:
                cash = cashflow
                defaults += 1
            else:
                cash = 0
        
        credit_ratings[i][index] = cash
    
    return credit_ratings
        

In [196]:
size = 10

In [197]:
cash_waterfall(1150, credit_ratings={"AAA":[0]*size, "AA":[0]*size, "A":[0]*size, "BBB":[0]*size, "Speculative":[0]*size})

{'AAA': [400.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'AA': [400.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'A': [350.0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'BBB': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'Speculative': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

## Monte-Carlo Simulation

In [17]:
rho = 0.1
sims = round(1e4)

In [18]:
while (rho <= 1):
    average = [0] * sims
    
    for x in range(sims):
        # forming the N-standard normal and its covariance matrix cov to form the correlated vector normal A
        N = np.random.normal(loc=0, scale=1, size=20)
        cov = np.reshape(np.array([1 if i==j else rho for j in range(20) for i in range(20)]),(20,20))
        A = N.dot(np.linalg.cholesky(cov))
        
        # create the default times with which to modify 
        U = ss.norm.cdf(A, loc=0, scale=1)
        T = -50*np.log(U)
        
        epv = 0
        maturity = 360
        credit_rating = {"AAA":[]*maturity, "AA":[]*maturity, "A":[]*maturity, "BBB":[]*maturity, "Speculative":[]*maturity}
        
        # 30 year maturity CDO being tranched
        for m in range(1,maturity+1):
            cash = sum([100 if (i > m/12) else 0 for i in T])
            epv += cash * np.math.exp(-0.03*m/12)
            
            
            
        average[x] = epv
    
    average_earn = np.average(average)
    
    print("At correlation parameter {} yeilded an average of {} in cashflow".format(rho, average_earn))
    print("The yeild to maturity is {}".format(YTM(average_earn,360,100)))
    print("Our cashflow-distribution is as follows")
    cash_waterfall(average_earn, ratings=credit_rating, type='yearly')
    print('===============================================================================================')
    rho += 0.1

At correlation parameter 0.1 yeilded an average of 23211.73517355558 in cashflow
The yeild to maturity is 3.165600000000003
Our cashflow-distribution is as follows
AAA CDO recieved 
4800.0

AA CDO recieved 
4800.0

A CDO recieved 
4800.0

BBB CDO recieved 
4800.0

Junk CDO recieved 
4011.74

At correlation parameter 0.2 yeilded an average of 22669.969386518536 in cashflow
The yeild to maturity is 3.3525000000000027
Our cashflow-distribution is as follows
AAA CDO recieved 
4800.0

AA CDO recieved 
4800.0

A CDO recieved 
4800.0

BBB CDO recieved 
4800.0

Junk CDO recieved 
3469.97

At correlation parameter 0.30000000000000004 yeilded an average of 21768.84167559846 in cashflow
The yeild to maturity is 3.6782000000000052
Our cashflow-distribution is as follows
AAA CDO recieved 
4800.0

AA CDO recieved 
4800.0

A CDO recieved 
4800.0

BBB CDO recieved 
4800.0

Junk CDO recieved 
2568.84

At correlation parameter 0.4 yeilded an average of 20679.392070069487 in cashflow
The yeild to maturit